In [ ]:
import pandas as pd 
import numpy as np 
from create_df import load_df, implement_column_creation
pd.set_option('display.max_columns', None)
df = load_df()
df = implement_column_creation(df)
df

/home/sam/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [12]:
df['vpip_players_before'] = df.apply(lambda row: vpip_players_before(row), axis=1)

In [11]:
import numpy as np 
def vpip_players_before(row):
    vpips = []
    vpip_dic = row['vpip_all_players'] # dictonary of vpips of each player
    players_before = row['players_acting_before_me'] # list of players acting before

    if players_before != {}: # skips if there are no players after
        for player in players_before: # look at all players after & find their vpip
            vpips.append(vpip_dic[player])
    if len(vpips) != 0:
        return np.mean(vpips)


In [10]:
players_after(df.HandHistory.iloc[-300])

{'P1_273288HX', 'P3_911389FK', 'P4_795240DR', 'P6_649055EH'}

In [13]:
def vpip_players_after()

,HandHistoryId,HandNumber,PokerSiteId,HandHistory,HandHistoryTimestamp,GameType,TournamentNumber,buyin,cash_or_tourn,my_blind_anti_total,my_cards,starting_stack,tournament_type,won,bet,total_players,position,table_max_players,bets_before_my_preflop_action,action_type,player_names,players_acting_before_me,players_acting_after_me,hour,year_month_day,days_since_start,hand_frequency,blinds,made_money,card_rank,BB,BB_in_stack,net_outcome,all_in,money_beyond_blind,outcome_relative_to_start,made_or_lost,prior_actions,vpip_all_players,flop_bet,turn_bet,river_bet,preflop_bet,high_card,low_card,suited,pocket_pair,gap,vpip_players_after,vpip_players_before
0,1,3951210633,1,"[<session sessioncode=""459742"">, <general>, <m...",2019-10-23 21:08:08,1,459742,1.10,Holdem NL,20.0,"[C5, H2]",1500.0,3-Table Sit & Go,0.0,20.0,9,0.222222,9,80.0,"{'P26_251842VD': 3, 'P4_131206CX': 0, 'P11_924...","[P18_572249IY, P14_459713MR, P25_871309MO, P3_...","[P26_251842VD, P4_131206CX, P11_924370QJ, P18_...",{P26_251842VD},21,2019-10-23,0,2155,"[10.0, 20.0]",0,0.5,20.0,75.00,-20.0,0.0,0,0.986667,0,"{'P18_572249IY': [], 'P14_459713MR': [], 'P25_...","{'P18_572249IY': 0, 'P14_459713MR': 0, 'P25_87...",0.0,0.0,0.0,0.0,5.0,2.0,0,0,3.0,0.000000,0.000000
1,2,3951211988,1,"[<session sessioncode=""459742"">, <general>, <m...",2019-10-23 21:09:34,1,459742,1.10,Holdem NL,10.0,"[C9, S9]",1480.0,3-Table Sit & Go,640.0,420.0,9,0.111111,9,40.0,"{'P4_131206CX': 0, 'P11_924370QJ': 0, 'P18_572...","[P18_572249IY, P14_459713MR, P25_871309MO, P3_...","[P4_131206CX, P11_924370QJ, P18_572249IY, P14_...","{P14_459713MR, P25_871309MO, P26_251842VD}",21,2019-10-23,0,2155,"[10.0, 20.0]",1,9.0,20.0,74.00,220.0,0.0,1,1.148649,1,"{'P18_572249IY': [0], 'P14_459713MR': [0], 'P2...","{'P18_572249IY': 0.0, 'P14_459713MR': 0.0, 'P2...",320.0,0.0,0.0,90.0,9.0,9.0,0,1,0.0,0.666667,0.142857
2,3,3951212830,1,"[<session sessioncode=""459742"">, <general>, <m...",2019-10-23 21:11:47,1,459742,1.10,Holdem NL,0.0,"[S3, DK]",1700.0,3-Table Sit & Go,0.0,0.0,9,1.000000,9,200.0,"{'P11_924370QJ': 0, 'P18_572249IY': 0, 'P14_45...","[P18_572249IY, P14_459713MR, P25_871309MO, P3_...","[P11_924370QJ, P18_572249IY, P14_459713MR, P25...","{P4_131206CX, P26_251842VD}",21,2019-10-23,0,2155,"[10.0, 20.0]",0,3.0,20.0,85.00,0.0,0.0,0,1.000000,1,"{'P18_572249IY': [0, 0], 'P14_459713MR': [0, 3...","{'P18_572249IY': 0.0, 'P14_459713MR': 0.5, 'P2...",0.0,0.0,0.0,0.0,13.0,3.0,0,0,10.0,0.250000,0.250000
3,4,3951214131,1,"[<session sessioncode=""459742"">, <general>, <m...",2019-10-23 21:13:37,1,459742,1.10,Holdem NL,0.0,"[C9, D8]",1700.0,3-Table Sit & Go,0.0,0.0,8,0.875000,9,0.0,"{'P18_572249IY': 0, 'P25_871309MO': 0, 'P3_286...","[P18_572249IY, P25_871309MO, P3_286963JX, P5_4...","[P18_572249IY, P25_871309MO, P3_286963JX, P5_4...","{P11_924370QJ, P4_131206CX, P26_251842VD}",21,2019-10-23,0,2155,"[10.0, 20.0]",0,5.5,20.0,85.00,0.0,0.0,0,1.000000,1,"{'P18_572249IY': [0, 0, 0], 'P25_871309MO': [2...","{'P18_572249IY': 0.0, 'P25_871309MO': 0.666666...",0.0,0.0,0.0,0.0,9.0,8.0,0,0,1.0,0.111111,0.250000
4,5,3951215154,1,"[<session sessioncode=""459742"">, <general>, <m...",2019-10-23 21:14:31,1,459742,1.10,Holdem NL,0.0,"[S7, HQ]",1700.0,3-Table Sit & Go,0.0,0.0,8,0.750000,9,0.0,"{'P25_871309MO': 0, 'P3_286963JX': 0, 'P5_4057...","[P18_572249IY, P25_871309MO, P3_286963JX, P5_4...","[P25_871309MO, P3_286963JX, P5_405793KB]","{P11_924370QJ, P18_572249IY, P4_131206CX, P26_...",21,2019-10-23,0,2155,"[10.0, 20.0]",0,2.0,20.0,85.00,0.0,0.0,0,1.000000,1,"{'P18_572249IY': [0, 0, 0, 0], 'P25_871309MO':...","{'P18_572249IY': 0.0, 'P25_871309MO': 0.5, 'P3...",0.0,0.0,0.0,0.0,12.0,7.0,0,0,5.0,0.187500,0.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54299,54333,4286799273,1,"[<session sessioncode=""3408296"">, <general>, <...",2021-03-30 08:12:25,82,41343764,5.25,Holdem NL,200

In [14]:
df.HandHistory.iloc[0]

['<session sessioncode="459742">',
 '<general>',
 '<mode>real</mode>',
 '<gametype>Holdem NL</gametype>',
 '<tablename>3-Table Sit &amp; Go (#30839441)</tablename>',
 '<duration>N/A</duration>',
 '<gamecount>0</gamecount>',
 '<startdate>2019-10-24 03:06:00</startdate>',
 '<currency>USD</currency>',
 '<nickname>Hero</nickname>',
 '<bets>0</bets>',
 '<wins>0</wins>',
 '<chipsin>0</chipsin>',
 '<chipsout>0</chipsout>',
 '<ipoints>0</ipoints>',
 '<statuspoints>0</statuspoints>',
 '<awardpoints>0</awardpoints>',
 '<tournamentname>3-Table Sit &amp; Go (#30839441)</tournamentname>',
 '<place>0</place>',
 '<buyin>$1+$0.10</buyin>',
 '<totalbuyin>$1.10</totalbuyin>',
 '<win>0</win>',
 '<maxplayers>9</maxplayers>',
 '<pokersite>Ignition</pokersite>',
 '</general>',
 '<game gamecode="3951210633">',
 '<general>',
 '<startdate>2019-10-24 03:08:08</startdate>',
 '<players>',
 '<player seat="5" name="P18_572249IY" chips="1500" dealer="0" win="0" bet="0" rebuy="0" addon="0" />',
 '<player seat="6" nam

In [28]:
def players_after(x):
    start = 0
    stop = 0
    hero_found = False
    players_before = set() # set of players before
    players_after = set()
    for idx, elem in enumerate(x): # list of players after 
        if 'Pocket' in elem:
            start = idx
    subset = x[start+1:] # subset containing preflop action until end
    for idx, elem in enumerate(subset):
        if '</round>' in elem:
            stop = idx
            break 
    subset = subset[:stop] # subset containing just preflop action 
    for elem in subset: 
        if 'Hero' in elem: # set to true and skip if hero in elem
            hero_found = True
            continue
        else:
            for sub_elem in elem.split(): 
                if 'player' in sub_elem:
                    player_name = sub_elem.replace('player="', '').replace('"', '') 
                    if hero_found == False: # if hero yet to act add to players before
                        players_before.add(player_name)
                    else:
                        if player_name in players_before: # check to see that player hasn't already acted
                            break
                        else:
                            players_after.add(player_name) # if player hasn't acted add to players after

    return players_after



In [26]:
df['players_acting_after_me'] = df.HandHistory.apply(lambda x: players_after(x)) # this might not account for cases where action was folded leading to some players never making an action. If we end up using data for hands where some players did not have to act we will have to develop a more robust function


In [27]:
df

,HandHistoryId,HandNumber,PokerSiteId,HandHistory,HandHistoryTimestamp,GameType,TournamentNumber,buyin,cash_or_tourn,my_blind_anti_total,my_cards,starting_stack,tournament_type,won,bet,total_players,position,table_max_players,bets_before_my_preflop_action,action_type,player_names,players_acting_before_me,players_acting_after_me,hour,year_month_day,days_since_start,hand_frequency,blinds,made_money,card_rank,BB,BB_in_stack,net_outcome,all_in,money_beyond_blind,outcome_relative_to_start,made_or_lost,prior_actions,vpip_all_players,flop_bet,turn_bet,river_bet,preflop_bet,high_card,low_card,suited,pocket_pair,gap,vpip_players_after,vpip_players_before
0,1,3951210633,1,"[<session sessioncode=""459742"">, <general>, <m...",2019-10-23 21:08:08,1,459742,1.10,Holdem NL,20.0,"[C5, H2]",1500.0,3-Table Sit & Go,0.0,20.0,9,0.222222,9,80.0,"{'P26_251842VD': 3, 'P4_131206CX': 0, 'P11_924...","[P18_572249IY, P14_459713MR, P25_871309MO, P3_...","[P26_251842VD, P4_131206CX, P11_924370QJ, P18_...",{},21,2019-10-23,0,2155,"[10.0, 20.0]",0,0.5,20.0,75.00,-20.0,0.0,0,0.986667,0,"{'P18_572249IY': [], 'P14_459713MR': [], 'P25_...","{'P18_572249IY': 0, 'P14_459713MR': 0, 'P25_87...",0.0,0.0,0.0,0.0,5.0,2.0,0,0,3.0,0.000000,0.000000
1,2,3951211988,1,"[<session sessioncode=""459742"">, <general>, <m...",2019-10-23 21:09:34,1,459742,1.10,Holdem NL,10.0,"[C9, S9]",1480.0,3-Table Sit & Go,640.0,420.0,9,0.111111,9,40.0,"{'P4_131206CX': 0, 'P11_924370QJ': 0, 'P18_572...","[P18_572249IY, P14_459713MR, P25_871309MO, P3_...","[P4_131206CX, P11_924370QJ, P18_572249IY, P14_...",{P26_251842VD},21,2019-10-23,0,2155,"[10.0, 20.0]",1,9.0,20.0,74.00,220.0,0.0,1,1.148649,1,"{'P18_572249IY': [0], 'P14_459713MR': [0], 'P2...","{'P18_572249IY': 0.0, 'P14_459713MR': 0.0, 'P2...",320.0,0.0,0.0,90.0,9.0,9.0,0,1,0.0,0.666667,0.142857
2,3,3951212830,1,"[<session sessioncode=""459742"">, <general>, <m...",2019-10-23 21:11:47,1,459742,1.10,Holdem NL,0.0,"[S3, DK]",1700.0,3-Table Sit & Go,0.0,0.0,9,1.000000,9,200.0,"{'P11_924370QJ': 0, 'P18_572249IY': 0, 'P14_45...","[P18_572249IY, P14_459713MR, P25_871309MO, P3_...","[P11_924370QJ, P18_572249IY, P14_459713MR, P25...","{P4_131206CX, P26_251842VD}",21,2019-10-23,0,2155,"[10.0, 20.0]",0,3.0,20.0,85.00,0.0,0.0,0,1.000000,1,"{'P18_572249IY': [0, 0], 'P14_459713MR': [0, 3...","{'P18_572249IY': 0.0, 'P14_459713MR': 0.5, 'P2...",0.0,0.0,0.0,0.0,13.0,3.0,0,0,10.0,0.250000,0.250000
3,4,3951214131,1,"[<session sessioncode=""459742"">, <general>, <m...",2019-10-23 21:13:37,1,459742,1.10,Holdem NL,0.0,"[C9, D8]",1700.0,3-Table Sit & Go,0.0,0.0,8,0.875000,9,0.0,"{'P18_572249IY': 0, 'P25_871309MO': 0, 'P3_286...","[P18_572249IY, P25_871309MO, P3_286963JX, P5_4...","[P18_572249IY, P25_871309MO, P3_286963JX, P5_4...","{P11_924370QJ, P4_131206CX, P26_251842VD}",21,2019-10-23,0,2155,"[10.0, 20.0]",0,5.5,20.0,85.00,0.0,0.0,0,1.000000,1,"{'P18_572249IY': [0, 0, 0], 'P25_871309MO': [2...","{'P18_572249IY': 0.0, 'P25_871309MO': 0.666666...",0.0,0.0,0.0,0.0,9.0,8.0,0,0,1.0,0.111111,0.250000
4,5,3951215154,1,"[<session sessioncode=""459742"">, <general>, <m...",2019-10-23 21:14:31,1,459742,1.10,Holdem NL,0.0,"[S7, HQ]",1700.0,3-Table Sit & Go,0.0,0.0,8,0.750000,9,0.0,"{'P25_871309MO': 0, 'P3_286963JX': 0, 'P5_4057...","[P18_572249IY, P25_871309MO, P3_286963JX, P5_4...","[P25_871309MO, P3_286963JX, P5_405793KB]","{P11_924370QJ, P18_572249IY, P4_131206CX, P26_...",21,2019-10-23,0,2155,"[10.0, 20.0]",0,2.0,20.0,85.00,0.0,0.0,0,1.000000,1,"{'P18_572249IY': [0, 0, 0, 0], 'P25_871309MO':...","{'P18_572249IY': 0.0, 'P25_871309MO': 0.5, 'P3...",0.0,0.0,0.0,0.0,12.0,7.0,0,0,5.0,0.187500,0.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54299,54333,4286799273,1,"[<session sessioncode=""3408296"">, <general>, <...",2021-03-30 08:12:25,82,41343764,5.25,Holdem NL,200.0,"[HA, H3]",1850.0,Hyper Turbo (500 Chip